### Chapter11 인터페이스 : 프로토콜에서 ABC까지

우리는 프로토콜을 '파이썬과 같은 동적 자료형 언어에서 다형성을 제공하는 비공식 인터페이스' 라고 정의했다. 또한 좀더 실용적으로는, 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스를 뜻하기도 한다. 따라서 프로토콜은 상속과는 무관하다. 그저 필요한 메서드만 갖춘다면 'XX 프로토콜' 이라고 부를 수 있는 것이다.

단 프로토콜은 '비공식 인터페이스'이기 때문에 공식 인터페이스처럼 어떤 메서드의 구현을 강제할 수 없다. 그것을 강제하기 위해서는 ABC를 사용해야 한다. 가령 시퀀스 프로토콜은 `__len__` 과 `__getitem__`을 요구하지만, 만약 간단한 반복만이 필요하다면 `__getitem__`만 구현하더라도 잘 작동한다. 또한 `__contains__` 를 구현하지 않더라도, 파이썬은 순차 검색을 통해서 `in`을 사용할 수 있게 해 준다. 이는 `abc.Sequence`를 상속하지 않은 클래스에 대해서도 마찬가지다. 상속과는 무관하다.

In [ ]:
class test:
  def __getitem__(self,pos):
    return range(0,5)[pos]

t=test()
print(t[3])
for i in t:
  print(i)

3
0
1
2
3
4


+ 런타임에 프로토콜 구현하기

멍키 패칭-코드를 실행하는 도중에 그 코드를 수정하는 것-은 언제나 좋은 것은 아니지만, 프로그램 중간에도 프로토콜을 동적으로 만들어 주는 데에 쓰일 수 있다. 이는 덕 타이핑의 예시이기도 한데, 해당 객체가 필요한 메서드를 '가지고 있기만 하면' 함수는 제대로 동작하는 것이다. 객체의 자료형은 아무 상관이 없다. 가령 `random.shuffle` 함수는 시퀀스 프로토콜과 `__setitem__` 특별 메서드를 요구하는데, 다음과 같이 동적으로 프로토콜을 구현해도 잘 작동한다.

In [5]:
import random

class numbers:
  nums=[1,2,3,4,5,6,7,8,9]

  def __len__(self):
    return len(self.nums)

  def __getitem__(self,pos):
    return self.nums[pos]

def set_num(self,pos,n):
  self.nums[pos]=n

numbers.__setitem__=set_num

ns=numbers()
random.shuffle(ns)
for i in ns:
  print(i)

3
1
2
4
5
9
7
8
6


+ ABC의 사용

ABC를 사용하는 것은 덕 타이핑을 약간은 벗어난다. 그러나 필요한 메서드만을 구현하는 것보다 ABC를 상속하는 것이 나을 때가 있을 수 있다. 물론 ABC를 직접 만드는 것은 웬만해서 지양해야 할 일이다. 너무 견고하게 만들 우려가 있고, 기존 ABC를 사용하는 것이 더 유리할 때가 훨씬 많기 때문이다(우리는 연습을 위해 ABC를 하나 만들지만). 가령 numbers.Integral 같은 것을 사용하면 `isinstance()` 를 사용할 때도 `int`를 쓰는 것보다 이점이 있다.

+ ABC 만들기

어떤 집합에서 원소를 무작위로 뽑아내지만, 들어 있는 원소를 다 뽑아내기 전까지는 같은 원소를 반복해 뽑아내면 안 되는 프레임워크를 만들어야 한다고 하자. 복권 당첨 번호를 뽑는 상자와 비슷하니 ABC의 이름을 `Tombola`로 한다.


In [7]:
import abc

class Tombola(abc.ABC):
  
  @abc.abstractmethod #추상 메서드를 뜻한다. 서브클래스에서도 이 데커레이터 붙은 메서드는 구현해야 함
  def load(self, iterable):
    """iterable 항목 추가"""

  @abc.abstractmethod
  def pick(self):
    """무작위로 항목 하나 제거하고 반환"""

  def loaded(self):
    """한 개 이상의 항목이 있으면 True, 아니면 False"""
    return bool(self.inspect())

  def inspect(self):
    """현재 남은 항목들로 구성된 튜플 반환"""
    items=[]
    while 1:
      try:
        items.append(self.pick())
      except LookupError:
        break
    self.load(items)
    return tuple(sorted(items)) #정렬된 형태로 반환한다

물론 `inspect()`메서드와 같이, 비효율적인 방식으로 작동한다고 생각되는 코드도 있다. 그러나 위 추상 클래스를 상속한 구상 클래스에서는 얼마든지 더 효율적인 방식으로 함수를 오버라이드할 수 있다. 핵심은 추상 클래스에서도 구상 메서드를 제공할 수 있다는 것이다. 단, 인터페이스에 정의된 다른 메서드들만을 사용해야 한다.

또한 ABC를 사용하는 것의 이점은, ABC를 상속한 구상 클래스에 특정 메서드의 구현을 강제할 수 있다는 것이다. 

In [8]:
class Fake(Tombola):
  def pick(self):
    return 10

f=Fake()

TypeError: ignored

위 코드를 실행시켜 보면, 우리가 Tombola에서 추상 메서드로 선언했던 `load()` 가 구현되어 있지 않기 때문에 Fake 클래스 객체를 생성할 때 에러가 발생하는 것을 알 수 있다.

+ abstractmethod 데커레이터

abc 모듈에서는 추상 메서드를 만들기 위한 `abc.abstractmethod` 데커레이터를 제공한다. 이때, 이 데커레이터는 제일 안쪽에 위치해야 한다. 즉 `@abstractmethod` 와 def문 사이에는 어떤 것도 올 수 없다.

+ Tombola ABC 상속하기

임의의 위치에 있는 공을 꺼내는 클래스를 Tombola 클래스 상속을 이용하여 만든다.

In [9]:
import random

class LotteryBlower(Tombola):

  def __init__(self,iterable):
    self._balls=list(iterable) # 무조건 리스트 형태로 저장한다. 코드의 융통성 향상

  def load(self,iterable):
    self._balls.extend(iterable)

  def pick(self):
    try:
      pos=random.randrange(len(self._balls))
    except:
      raise LookupError('pick from empty BingoCage')
    return self._balls.pop(pos)

  def loaded(self):
    return bool(self._balls)
  
  def inspect(self):
    return tuple(sorted(self._balls))

+ 가상 서브클래스

이때 어떤 클래스가 ABC를 명시적으로 상속하지 않더라도 ABC의 가상 서브클래스로 등록하는 방법이 있다. register() 메서드를 사용하거나 ABC.register 데커레이터를 사용하면 된다. 그러면 등록된 클래스는 issubclass() 와 isinstance() 에 의해 인식된다.

단 그렇게 가상 서브클래스로 등록해 놓고 ABC에 정의된 인터페이스를 충실하게 구현하지 않으면 런타임 에러가 발생한다.

+ 클래스 등록 없는 가상 서브클래스

register() 등을 이용해 클래스를 등록하지 않고도 ABC의 가상 서브클래스로 인식시킬 수 있는 경우가 있다. 가령 `__len__` 을 구현한 클래스는 abc.Sized 의 서브클래스로 취급된다.

In [11]:
class LenTest:
  def __len__(self):return 1

from collections import abc
print(isinstance(LenTest(), abc.Sized))

True


이는 `abc.Sized` 가 `__subclasshook__` 이라는 특별 클래스 메서드를 구현하기 때문이다. 이 메서드는 클래스 자체나 클래스가 상속한 슈퍼클래스들 중 `__len__` 을 구현한 클래스가 있으면 True를 반환한다. 따라서 `__subclasshook__` 메서드를 구현할 시, 특정 메서드들을 가진 클래스가 있으면 자동적으로 그게 특정 ABC의 가상 서브클래스가 되도록 만들 수도 있다. 그러나 대부분의 경우, 그건 좋은 선택이 아니다.